In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Сиды

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import random

seed=42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)

In [ ]:
#device

# Импорты

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


# TEST MODE

In [ ]:
TEST_MODE=True

# Загрузка данных

In [ ]:
items=pd.read_parquet('/kaggle/input/vseros-ai-dzen/items.parquet')
train=pd.read_csv('/kaggle/input/vseros-ai-dzen/train (12).csv')
test=pd.read_csv('/kaggle/input/vseros-ai-dzen/test (8).csv')
sample=pd.read_csv('/kaggle/input/vseros-ai-dzen/sample_submission (5).csv')

## text_target2num

In [ ]:
#text_target2num={lg: i for i, lg in enumerate(sorted(train['target'].unique()))}
#num2text_target={i: lg for lg, i in text_target2num.items()}

In [ ]:
'''# делаем стабильный маппинг только из TRAIN
classes = sorted(pairs_train["target"].dropna().unique().tolist())
cls2id = {c:i for i,c in enumerate(classes)}
id2cls = {i:c for c,i in cls2id.items()}
num_labels = len(classes)

train = train[train["target"].isin(classes)].copy()
train["target"] = train["target"].map(cls2id).astype(int)

# safety: никаких NaN/вылетов
assert train["label_id"].between(0, num_labels-1).all()
'''

In [ ]:
items

In [ ]:
train

# Merge

In [ ]:
import pandas as pd

# 1) Load
items = pd.read_parquet("/kaggle/input/vseros-ai-dzen/items.parquet")[["itemId","authorId","title","content"]]
pairs_train = pd.read_csv("/kaggle/input/vseros-ai-dzen/train (12).csv", dtype={"leftItemId":str, "rightItemId":str, "target":str})






if TEST_MODE:
    pairs_train=pairs_train[:55000]
else:
    pairs_train=pairs_train





# 2) Make sure keys are strings (protect against overflow)
items["itemId"] = items["itemId"].astype(str)

# 3) Deduplicate items if needed
items = items.drop_duplicates("itemId", keep="last").set_index("itemId")

# 4) Prepare left/right views and join
left_meta  = items.rename(columns=lambda c: f"left_{c}")    # columns: left_authorId,left_title,left_content
right_meta = items.rename(columns=lambda c: f"right_{c}")   # columns: right_authorId,right_title,right_content

train = (pairs_train
      .join(left_meta,  on="leftItemId")
      .join(right_meta, on="rightItemId"))

# (optional) keep a tidy column order
cols = ["left_title","right_title" ,"target",
        ]
train = train[cols]
#train['target']=train['target'].map(text_target2num)


In [ ]:
pairs_test = pd.read_csv("/kaggle/input/vseros-ai-dzen/test (8).csv", dtype={"leftItemId":str, "rightItemId":str, "target":str})

test = (pairs_test
      .join(left_meta,  on="leftItemId")
      .join(right_meta, on="rightItemId"))

# (optional) keep a tidy column order
cols = ["left_title","right_title" ,
        ]
test = test[cols]

#test['target']=test['target'].map(text_target2num)


# Маппинг

In [ ]:

WANTED = ['no_relevant','relevant_minus','relevant','relevant_plus', 'doubles']

train = train[train['target'].isin(WANTED)].copy()

cls2id  = {c:i for i,c in enumerate(WANTED)}  
id2cls  = {i:c for c,i in cls2id.items()}
num_labels = len(WANTED)  # 4

train['target'] = train['target'].map(cls2id).astype('int64')



In [ ]:
train['target'].unique()

In [ ]:
train_data, eval_data = train_test_split(
    train, test_size=0.2, stratify=train['target'], random_state=seed
)

# Dataset

In [ ]:
class DzenDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, has_label=True):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.has_label = has_label

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]

        enc = self.tokenizer(
            r["left_title"] or "",
            r["right_title"] or "",
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        item = {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
        }
        
        if "token_type_ids" in enc:
            item["token_type_ids"] = enc["token_type_ids"].squeeze(0)

        if self.has_label:
            item["labels"] = torch.tensor(int(r["target"]), dtype=torch.long)

        return item


# Model

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=5,)

In [ ]:
#xlm-roberta-large deepvk/USER-bge-m3

# Tokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-large')
#ai-forever/FRIDA

# Создание датасетов

In [ ]:
train_dataset=DzenDataset(train_data, tokenizer, max_len=128, has_label=True )
eval_dataset=DzenDataset(eval_data, tokenizer, max_len=128, has_label=True)

test_dataset=DzenDataset(test, tokenizer, max_len=128,  has_label=False)


# TrainingArguments

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments, SentenceTransformerTrainer

In [ ]:
args=TrainingArguments(
    output_dir='goal',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    learning_rate=3e-6,
    lr_scheduler_type='cosine',
    eval_strategy='epoch',
    report_to='none',
    #logging_strategy="steps",
    #save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    weight_decay=0.02,
    warmup_ratio=0.06,
    fp16=True,
    #max_grad_norm=1.0,
    label_smoothing_factor=0.025
)

# Compute Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_macro":    f1_score(labels, preds, average="macro",    zero_division=0),
        "accuracy":    (preds == labels).mean(),
    }


# Trainer

In [ ]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

# Train

In [ ]:
trainer.train()

# Submission

In [ ]:
pred= trainer.predict(test_dataset)
test_pred_ids = pred.predictions.argmax(-1)
test_pred_text = [id2cls[i] for i in test_pred_ids]

submission = sample.copy()
submission["target"] = test_pred_text
submission.to_csv("submission4.csv", index=False)